In [4]:
import os
os. chdir('/home/users/gmenon/workspace/songsLyricsGenerator/src')

In [5]:
import torch
from torch import nn
from pytorch_lightning import LightningModule
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model
from constants.mir_constants import TrainingArgs, WAV2VEC2_ARGS
from training import lyrics_finetune
from dataclasses import dataclass, asdict
import json
import argparse
from dataclasses import dataclass

wandb: Currently logged in as: greeshmasmenon (songslyricstranscription). Use `wandb login --relogin` to force relogin


{
    "TRAIN_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_song_metadata_en_demucs_cleaned.csv",
    "TEST_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_song_metadata_en_demucs_cleaned.csv",
    "MODEL_BACKBONE": "facebook/wav2vec2-large-960h-lv60-self",
    "BATCH_SIZE": 2,
    "NUM_EPOCHS": 15,
    "MODEL_SAVE_PATH": "/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt",
    "FINETUNE_STRATEGY": "no_freeze_deepspeed",
    "LR_SCHEDULER": "reduce_on_plateau_schedule"
}


In [ ]:
print(json.dumps(asdict(WAV2VEC2_ARGS), indent = 4))

In [ ]:
import flash
import torchaudio
import pytorch_lightning as pl

# class SpeechDataModule(pl.LightningDataModule):
#     def __init__(self, args: WAV2VEC2_ARGS):
#         super().__init__()
#         self.batch_size = 2

#     def prepare_data(self):
#         # Download and extract the audio files and transcripts, if necessary.
#         pass

#     def setup(self, stage=None):
#         # Create the dataset
#         # datamodule = 
#         self.dataset = SpeechRecognitionData.from_csv("consolidated_file_path",
#                                                          "transcription_capitalized",
#                                                          train_file=WAV2VEC2_ARGS.TRAIN_FILE_PATH,
#                                                          test_file=WAV2VEC2_ARGS.TEST_FILE_PATH,
#                                                          batch_size=2
#                                                          )

#         # Create the dataloader
#         self.train_dataloader = self.dataset.train_dataloader
#         # torch.utils.data.DataLoader(
#         #     self.dataset,
#         #     batch_size=self.batch_size,
#         #     shuffle=True,
#         #     num_workers=4,
#         # )
#         self.val_dataloader = self.dataset.val_dataloader
#         # torch.utils.data.DataLoader(
#         #     self.dataset,
#         #     batch_size=self.batch_size,
#         #     shuffle=False,
#         #     num_workers=4,
#         # )

#     def train_dataloader(self):
#         return self.train_dataloader

#     def val_dataloader(self):
#         return self.val_dataloader

In [88]:
# class DALIDataset(pl.LightningDataModule):
#     def __init__(self, batch_size: int = 4,
#                   train_path :Optional[str] = None,
#                     validation_path: Optional[str] = None,
#                       model_backbone: pl.LightningModule = None,
#                       args: TrainingArgs = WAV2VEC2_ARGS
#                       ):
        
#         super().__init__()
#         self.train_path = train_path if train_path is not None else args.TRAIN_FILE_PATH
#         self.validation_path = validation_path if validation_path is not None else args.TEST_FILE_PATH
#         self.model_backbone = model_backbone if model_backbone is not None else args.MODEL_BACKBONE

#         def prepare_data(self):
#             pass
        
#         def setup(self):
#             train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH) 
#             validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)
#             songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
#             audio_dataset = Dataset.from_dict(
#                 {"audio": list(songs_metadata["file_name"]),
#                  "transcription": list(songs_metadata["transcription"])}).cast_column("audio", Audio(sampling_rate=16_000))
#             audio_dataset["transcription"] = audio_dataset["transcription"] = re.sub(WAV2VEC2_ARGS.CHARS_TO_REMOVE_FROM_TRANSCRIPTS, '', audio_dataset["transcription"]).upper()
#             audio_dataset = audio_dataset.train_test_split(test_size=0.2, shuffle=True)
            



In [89]:
# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
# from datasets import load_dataset,Dataset,Audio
# import pandas as pd
# import re
# import os

# class SpeechRecognitionDataset(Dataset):
#     def __init__(self, dataset,args:TrainingArgs,processor):
#         self.model_backbone = args.MODEL_BACKBONE
#         self.dataset = dataset
#         self.processor = processor
#         self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(self.model_backbone)
#         self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(self.model_backbone)
#         self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        
#     def __getitem__(self, index):
#         audio = self.dataset[index]["audio"][0]
#         transcription = self.dataset[index]["transcription"][0]

#         input_values = self.processor(audio["array"], sampling_rate=16_000).input_values[0]
#         with self.processor.as_target_processor():
#             print("Entering the label encoder")
#             labels = self.processor(transcription,return_tensors = 'pt').input_ids
#        # attention_mask = inputs.attention_mask.squeeze()
#         return input_values, labels
    
#     def __len__(self):
#         return len(self.dataset)



In [90]:
# from typing import Union, List, Dict
# import torch

# from dataclasses import dataclass, field
# from typing import Any, Dict, List, Optional, Union


# class DataCollatorCTCWithPadding:
#     """
#     Data collator that will dynamically pad the inputs received.
#     Args:
#         processor (:class:`~transformers.Wav2Vec2Processor`)
#             The processor used for proccessing the data.
#         padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
#             Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
#             among:
#             * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
#               sequence if provided).
#             * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
#               maximum acceptable input length for the model if that argument is not provided.
#             * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
#               different lengths).
#         max_length (:obj:`int`, `optional`):
#             Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
#         max_length_labels (:obj:`int`, `optional`):
#             Maximum length of the ``labels`` returned list and optionally padding length (see above).
#         pad_to_multiple_of (:obj:`int`, `optional`):
#             If set will pad the sequence to a multiple of the provided value.
#             This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
#             7.5 (Volta).
#     """

#     processor: Wav2Vec2Processor
#     padding: Union[bool, str] = True
#     max_length: Optional[int] = None
#     max_length_labels: Optional[int] = None
#     pad_to_multiple_of: Optional[int] = None
#     pad_to_multiple_of_labels: Optional[int] = None

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # split inputs and labels since they have to be of different lengths and need
#         # different padding methods
#         input_features = [{"input_values": feature["input_values"]} for feature in features]
#         label_features = [{"input_ids": feature["labels"]} for feature in features]

#         batch = self.processor.pad(
#             input_features,
#             padding=self.padding,
#             max_length=self.max_length,
#             pad_to_multiple_of=self.pad_to_multiple_of,
#             return_tensors="pt",
#         )
#         with self.processor.as_target_processor():
#             labels_batch = self.processor.pad(
#                 label_features,
#                 padding=self.padding,
#                 max_length=self.max_length_labels,
#                 pad_to_multiple_of=self.pad_to_multiple_of_labels,
#                 return_tensors="pt",
#             )

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         batch["labels"] = labels

#         return batch

In [91]:
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained("bert-base-uncased")
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)

In [6]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [10]:
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor,BertTokenizer
from datasets import load_dataset,Dataset,Audio
import pandas as pd
import re
import os

class SpeechRecognitionDataModule(pl.LightningDataModule):
    def __init__(self, WAV2VEC2_ARGS: WAV2VEC2_ARGS, num_workers):
        super().__init__()
        self.batch_size = WAV2VEC2_ARGS.BATCH_SIZE
        self.num_workers = num_workers
        self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(WAV2VEC2_ARGS.MODEL_BACKBONE)
        #self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.bert_tokenizer)
        self.data_collator = DataCollatorCTCWithPadding(processor=self.processor, padding=True)
        self.cpu_count = 4
        print(f"CPU Count = {self.cpu_count}")
    
    def setup(self, stage=None):
        train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH)
        validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)
        #songs_metadata = pd.concat([train_df,validation_df], ignore_index = True)
        if stage == 'fit' or stage is None:
            print("In Stage = Fit")
            train_dataset = Dataset.from_dict(
                    {"audio": list(train_df["consolidated_file_path"]),
                    "transcription": list(train_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.train_dataset = train_dataset.map(self.prepare_dataset,remove_columns = train_dataset.column_names)
            
            val_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            self.val_dataset = val_dataset.map(self.prepare_dataset,remove_columns = val_dataset.column_names)
            #common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)

        
        if stage == 'test' or stage is None:
            print("In Stage = Test")
            test_dataset = Dataset.from_dict(
                    {"audio": list(validation_df["consolidated_file_path"]),
                    "transcription": list(validation_df["transcription_capitalized"])}).cast_column("audio", Audio(sampling_rate=16_000))
            test_dataset = val_dataset.map(self.prepare_dataset,remove_columns = val_dataset.column_names)
            self.test_dataset = test_dataset
    
    def train_dataloader(self):
        print("entering train data loader")
        return DataLoader(
            self.train_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )
    
    def val_dataloader(self):
        print("entering val data loader")
        return DataLoader(
            self.val_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )
    
    def test_dataloader(self):
        print("entering test data loader")
        return DataLoader(
            self.test_dataset.with_format("torch"), 
            batch_size=self.batch_size, 
            num_workers=self.num_workers,
            collate_fn = self.data_collator
        )
        
    # def prepare_dataset(self, batch):
    #     audio = batch["audio"]
    #     transcription = batch["transcription"]
    #     #batch["input_values"] = audio["array"]
    #     batch["input_values"] = self.feature_extractor(audio["array"],sampling_rate=16_000)
    #     #batch["labels"] = self.bert_tokenizer(transcription, padding=True, truncation=True).input_ids
    #     batch["labels"]=transcription
    #     #batch["audios"] = audio["array"]
    #     # batch["input_length"] = len(batch["input_values"])
    #     # batch["target_length"] = len(batch["labels"])
    #     return batch
    # #tokenizer(batch["target_text"], **additional_kwargs).input_ids

    def prepare_dataset(self,batch):
        audio = batch["audio"]
        # batched output is "un-batched" to ensure mapping is correct
        batch["input_values"] = self.processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
        batch["input_length"] = len(batch["input_values"])
        with self.processor.as_target_processor():
            batch["labels"] = self.processor(batch["transcription"]).input_ids
        return batch

In [30]:
import torch
from torch import nn
from pytorch_lightning import LightningModule, Trainer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC

class Wav2SeqModel(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.batch_size = hparams.batch_size
        # self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(hparams.wav2vec2_model)
        # self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(hparams.wav2vec2_model)
        # self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.wav2vec2 = AutoModelForCTC.from_pretrained(hparams.wav2vec2_model,ctc_zero_infinity=True,ctc_loss_reduction="mean")
        self.wav2vec2.freeze_feature_encoder()
        print(self.wav2vec2.config)
        self.seq2seq = AutoModelForCausalLM.from_pretrained(hparams.lm_model)
        self.seq2seq.config.is_decoder = True
        self.enc_to_dec_proj = nn.Linear(self.wav2vec2.config.hidden_size, self.seq2seq.config.hidden_size)
        print(self.seq2seq.config)
        #self.data_collator = DataCollatorCTCWithPadding()


    def forward(self, audio, labels):
        #print("entering forward step")
        encoder_outputs = self.wav2vec2(audio[0],output_hidden_states=True,output_attentions=True)
        # print(f"encoder_outputs = {encoder_outputs[0].shape},{encoder_outputs[0]},logits = {encoder_outputs.logits},hidden_states = {encoder_outputs.hidden_states}")
        # encoder_attention_mask = self.wav2vec2._get_feature_vector_attention_mask(
        #         encoder_outputs[0].shape[1], encoder_outputs.attentions)
        #print(f"encoder_attention_mask = {encoder_outputs.attentions}")
        predicted_ids = torch.argmax(encoder_outputs.logits, dim=-1)
        predicted_ids = predicted_ids[:,:512]
        # print(predicted_ids.shape)
        encoder_hidden_states = encoder_outputs[0]
        # print(encoder_hidden_states.shape,type(encoder_hidden_states),encoder_hidden_states)
        # print(f"pad_token_id = {self.seq2seq.config.pad_token_id}")
        
        #x_hidden_states = self.enc_to_dec_proj(x.hidden_states[0])
        #print(f"hidden states = {x.hidden_states[0].shape},{x.hidden_states[0]}")
        # print(f"attentions = {len(x.attentions[0])}, {x.attentions}")
        # print(f"logits = {len(x.logits[0])},{x.logits}")
        decoder_input_ids = self.shift_tokens_right(labels, 0, 0)
        decoder_outputs = self.seq2seq(input_ids=decoder_input_ids,encoder_hidden_states=encoder_hidden_states) #,attention_mask = x.attentions, decoder_input_ids=predicted_ids

            #     decoder_outputs = self.decoder(
            # input_ids=decoder_input_ids,
            # attention_mask=decoder_attention_mask,
            # encoder_hidden_states=encoder_hidden_states,
            # encoder_attention_mask=encoder_attention_mask,
            # inputs_embeds=decoder_inputs_embeds,
            # output_attentions=output_attentions,
            # output_hidden_states=output_hidden_states,
            # use_cache=use_cache,
            # past_key_values=past_key_values,
            # return_dict=return_dict,
        return decoder_outputs

    def generate(self, audio):
        encoder_outputs = self.wav2vec2(audio[0],output_hidden_states=True,output_attentions=True)
        encoder_hidden_states = encoder_outputs[0]
        #tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        bos_ids = (
            torch.ones(
                (encoder_hidden_states.size()[0], 1),
                dtype=torch.long,
                device=self.seq2seq.device,
            )
            * self.seq2seq.config.pad_token_id
        )
        return self.seq2seq.generate(
            input_ids=bos_ids,
            encoder_hidden_states=encoder_hidden_states,
        )

    def training_step(self, batch, batch_idx):
        #print("entering training step")
        # audio = batch["input_values"].unsqueeze(0)
        # labels = batch["labels"]
        # labels = labels.reshape(-1)
        # # input_length = batch["input_length"]
        # # target_length = batch["input_length"]
        # #audio,labels,attention_mask = batch
        # # audio = batch["input_values"]
        # # labels = batch["labels"]
        # # print(audio)
        # # print(labels)
        # logits = self(audio)
        # logits = logits.reshape(-1, self.seq2seq.config.vocab_size)
        # input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        # target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        # ctc_loss =  nn.CTCLoss(blank=0)
        # loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        # self.log('train_loss', loss, on_step=True, on_epoch=True)
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        labels = labels.reshape(self.batch_size,-1)
        # input_length = batch["input_length"]
        # target_length = batch["input_length"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio,labels).logits
        logits = logits.reshape(-1, self.batch_size, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        #print(logits.shape,labels.shape,input_lengths.shape,target_lengths.shape)
        #print(f"input_lengths = {input_lengths}")
        #print(f"target_lengths = {target_lengths}")
        #loss = nn.CTCLoss(blank=0).forward(logits, labels,input_length,target_length)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        #loss_fct = nn.CrossEntropyLoss()
        #loss = loss_fct(logits.reshape(-1, self.seq2seq.config.vocab_size), labels.reshape(-1))
        return loss

    def validation_step(self, batch,batch_idx):
        #print("entering validation step")
       # print(batch)
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        labels = labels.reshape(self.batch_size,-1)
        # input_length = batch["input_length"]
        # target_length = batch["input_length"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio,labels).logits
        logits = logits.reshape(-1, self.batch_size, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        # print(logits.shape,labels.shape,input_lengths.shape,target_lengths.shape)
        # print(f"input_lengths = {input_lengths}")
        # print(f"target_lengths = {target_lengths}")
        #loss = nn.CTCLoss(blank=0).forward(logits, labels,input_length,target_length)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        #loss_fct = nn.CrossEntropyLoss()
        #loss = loss_fct(logits.reshape(-1, self.seq2seq.config.vocab_size), labels.reshape(-1))
        self.log('val_loss', loss, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=hparams.learning_rate)

    @staticmethod
    def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
        """
        Shift input ids one token to the right.
        """
        shifted_input_ids = input_ids.new_zeros(input_ids.shape)
        shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
        if decoder_start_token_id is None:
            raise ValueError("Make sure to set the decoder_start_token_id attribute of the model's configuration.")
        shifted_input_ids[:, 0] = decoder_start_token_id
    
        if pad_token_id is None:
            raise ValueError("Make sure to set the pad_token_id attribute of the model's configuration.")
        # replace possible -100 values in labels by `pad_token_id`
        shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)
    
        return shifted_input_ids

# {'input': tensor([[-0.0004, -0.0008, -0.0004,  ..., -0.0038, -0.0031,  0.0000]],
#        device='cuda:0'), 'target': ['SOMETHING IS NOT THE SAME'], <DataKeys.METADATA: 'metadata'>: [{'sampling_rate': 16000}]}


    # def validation_step(self, batch, batch_idx):
    #     x, y = batch
    #     y_hat = self.model(x)
    #     loss = F.cross_entropy(y_hat, y)
    #     self.log("val_loss", loss)


In [33]:
test_data = SpeechRecognitionDataModule(WAV2VEC2_ARGS, num_workers = 0)
test_data.setup()


CPU Count = 4
In Stage = Fit


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In Stage = Test


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [31]:

import pytorch_lightning as pl
hparams = argparse.Namespace()
hparams.wav2vec2_model = 'facebook/wav2vec2-large-960h-lv60-self'
hparams.lm_model = 'bert-base-uncased' #'facebook/bart-large'
hparams.vocab_size = 10000
hparams.learning_rate = 0.00001
hparams.batch_size = WAV2VEC2_ARGS.BATCH_SIZE

model = Wav2SeqModel(hparams)
trainer = Trainer(max_epochs=1,devices=1, accelerator="gpu")
trainer.fit(model,SpeechRecognitionDataModule(WAV2VEC2_ARGS,num_workers=4))

# {'audio': [{'path': None, 'array': array([-0.05114746, -0.11273193, -0.09152222, ...,  0.2796936 ,
#         0.29998779,  0.20550537]), 'sampling_rate': 16000}, {'path': None, 'array': array([ 0.14523315,  0.05508423, -0.13534546, ...,  0.01959229,
#         0.00973511, -0.00811768]), 'sampling_rate': 16000}], 'transcription': ["just a tryin' to survive", "don't go on me"], 'input_values':

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-960h-lv60-self",
  "activation_dropout": 0.1,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": true,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dr

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

CPU Count = 4
In Stage = Fit


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

...

entering val data loader
entering train data loader


/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [32]:
import librosa 
def create_audio_tensor(audio_path):
    audio, sample_rate = librosa.load(audio_path)
    audio_tensor = torch.from_numpy(audio).float()
    audio_tensor = audio_tensor.unsqueeze(0)
    return audio_tensor
audio_tensor = create_audio_tensor("notebooks/separated/mdx_extra/test_clip/vocals.wav")
print(audio_tensor.unsqueeze(0).shape)
model.eval()
with torch.no_grad():
    y_hat = model.generate(audio_tensor.unsqueeze(0))
torch.argmax(y_hat, dim=-1)

torch.Size([1, 1, 47149])


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


IndexError: tuple index out of range

In [ ]:
y_hat

In [34]:
audio_tensor = next(iter(test_data.val_dataloader()))["input_values"]
audio_tensor

entering val data loader


tensor([[ 0.0164,  0.3489,  0.2281,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1105, -0.5085, -0.7563,  ..., -0.0009, -0.0591, -0.0010]])

In [35]:
model.eval()
with torch.no_grad():
    y_hat = model.generate(audio_tensor.unsqueeze(0))
torch.argmax(y_hat, dim=-1)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/mnt/irisgpfs/users/gmenon/workspace/songsLyricsGenerator/slg_wav2vec2/lib/python3.8/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


IndexError: tuple index out of range

In [30]:
# import librosa
# import torch
# from transformers import Wav2Vec2ForCTC

# def create_audio_tensor(audio_path):
#     audio, sample_rate = librosa.load(audio_path)
#     audio_tensor = torch.from_numpy(audio).float()
#     audio_tensor = audio_tensor.unsqueeze(0)
#     return audio_tensor

# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
# audio_tensor = create_audio_tensor("notebooks/separated/mdx_extra/test_clip/vocals.wav")
logits = model(audio_tensor).logits
predictions = torch.argmax(logits, dim=-1)
print(audio_tensor)

tensor([[ 0.0164,  0.3489,  0.2281,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1105, -0.5085, -0.7563,  ..., -0.0009, -0.0591, -0.0010]])


In [31]:
predictions

tensor([[18, 11,  0,  0,  0,  0,  0,  7,  0,  0,  0,  0,  6,  0,  0,  0,  4,  4,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  0,  0,  0,
          0, 15, 15,  0,  0,  0,  0,  0,  0, 12,  0,  0,  0,  4,  4,  0,  0,  0,
         14,  0,  0,  0,  0, 10,  0,  0, 12, 12,  0,  0,  0,  0,  0, 19,  0,  0,
          0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6, 11,
          0,  0,  5,  0,  0,  0,  4,  4,  0,  0,  0, 24,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  5,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  4,  4,  4,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 19,  0,  0,  0,  0,  0,  0,  0,  0,
          7,  0,  0,  0,  0, 23,  0,  5,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,
          5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 15,  0,  0,  0,  0,  0,  0,
      

In [78]:
m = nn.LogSoftmax(dim=1)
input = torch.randn(2, 3)
output = m(input)

In [79]:
input,output

(tensor([[ 0.5917, -0.8438, -0.5997],
         [-0.7059,  0.3278,  1.0544]]),
 tensor([[-0.4329, -1.8685, -1.6244],
         [-2.2644, -1.2308, -0.5041]]))

In [ ]:
import torch
from torch import nn
from pytorch_lightning import LightningModule, Trainer
from transformers import Wav2Vec2ForCTC, Wav2Vec2Model, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC

class Wav2SeqModel(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.batch_size = hparams.batch_size
        self.save_hyperparameters()
        # self.tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(hparams.wav2vec2_model)
        # self.feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(hparams.wav2vec2_model)
        # self.processor = Wav2Vec2Processor(feature_extractor=self.feature_extractor, tokenizer=self.tokenizer)
        self.wav2vec2 = AutoModelForCTC.from_pretrained(hparams.wav2vec2_model,ctc_zero_infinity=True,ctc_loss_reduction="sum")
        self.wav2vec2.freeze_feature_encoder()
        # print(self.wav2vec2.config)
        self.seq2seq = AutoModelForCausalLM.from_pretrained(hparams.lm_model)
        self.enc_to_dec_proj = nn.Linear(self.wav2vec2.config.hidden_size, self.seq2seq.config.hidden_size)
        print(self.seq2seq.config)
        #self.data_collator = DataCollatorCTCWithPadding()


    def forward(self, audio):
        print("entering forward step")
        x = self.wav2vec2(audio[0],output_hidden_states=True,output_attentions=True)
        predicted_ids = torch.argmax(x.logits, dim=-1)
        #x_ids = self.enc_to_dec_proj(predicted_ids)
        #x_hidden_states = self.enc_to_dec_proj(x.hidden_states[0])
        #print(f"hidden states = {x.hidden_states[0].shape},{x.hidden_states[0]}")
        # print(f"attentions = {len(x.attentions[0])}, {x.attentions}")
        # print(f"logits = {len(x.logits[0])},{x.logits}")
        logits = self.seq2seq(input_ids = predicted_ids).logits #,attention_mask = x.attentions, decoder_input_ids=predicted_ids

            #     decoder_outputs = self.decoder(
            # input_ids=decoder_input_ids,
            # attention_mask=decoder_attention_mask,
            # encoder_hidden_states=encoder_hidden_states,
            # encoder_attention_mask=encoder_attention_mask,
            # inputs_embeds=decoder_inputs_embeds,
            # output_attentions=output_attentions,
            # output_hidden_states=output_hidden_states,
            # use_cache=use_cache,
            # past_key_values=past_key_values,
            # return_dict=return_dict,
        return logits

    def training_step(self, batch, batch_idx):
        print("entering training step")
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        labels = labels.reshape(-1)
        # input_length = batch["input_length"]
        # target_length = batch["input_length"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio)
        logits = logits.reshape(-1, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch,batch_idx):
        print("entering validation step")
        print(batch)
        audio = batch["input_values"].unsqueeze(0)
        labels = batch["labels"]
        labels = labels.reshape(-1)
        # input_length = batch["input_length"]
        # target_length = batch["input_length"]
        #audio,labels,attention_mask = batch
        # audio = batch["input_values"]
        # labels = batch["labels"]
        # print(audio)
        # print(labels)
        logits = self(audio)
        logits = logits.reshape(-1, self.seq2seq.config.vocab_size)
        input_lengths = torch.full(size=(self.batch_size,), fill_value=logits.shape[0], dtype=torch.long)
        target_lengths = torch.full(size=(self.batch_size,), fill_value=labels.shape[0], dtype=torch.long)
        print(logits.shape,labels.shape,input_lengths.shape,target_lengths.shape)
        #loss = nn.CTCLoss(blank=0).forward(logits, labels,input_length,target_length)
        ctc_loss =  nn.CTCLoss(blank=0)
        loss = ctc_loss(logits,labels,input_lengths,target_lengths)
        #loss_fct = nn.CrossEntropyLoss()
        #loss = loss_fct(logits.reshape(-1, self.seq2seq.config.vocab_size), labels.reshape(-1))
        self.log('val_loss', loss, on_step=True, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=hparams.learning_rate)

# {'input': tensor([[-0.0004, -0.0008, -0.0004,  ..., -0.0038, -0.0031,  0.0000]],
#        device='cuda:0'), 'target': ['SOMETHING IS NOT THE SAME'], <DataKeys.METADATA: 'metadata'>: [{'sampling_rate': 16000}]}


    # def validation_step(self, batch, batch_idx):
    #     x, y = batch
    #     y_hat = self.model(x)
    #     loss = F.cross_entropy(y_hat, y)
    #     self.log("val_loss", loss)


In [ ]:
train_df = pd.read_csv(WAV2VEC2_ARGS.TRAIN_FILE_PATH)
validation_df = pd.read_csv(WAV2VEC2_ARGS.TEST_FILE_PATH)

In [ ]:
train_df[train_df.transcription_capitalized.str.len()>200]

# Readying the codebase

In [1]:
import os
os. chdir('/home/users/gmenon/workspace/songsLyricsGenerator/src')
from training import lyrics_finetune
from constants.mir_constants import TrainingArgs, WAV2VEC2_ARGS

wandb: Currently logged in as: greeshmasmenon (songslyricstranscription). Use `wandb login --relogin` to force relogin


{
    "TRAIN_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/train_song_metadata_en_demucs_cleaned.csv",
    "TEST_FILE_PATH": "/home/users/gmenon/notebooks/home/users/gmenon/notebooks/validation_song_metadata_en_demucs_cleaned.csv",
    "MODEL_BACKBONE": "facebook/wav2vec2-large-960h-lv60-self",
    "BATCH_SIZE": 2,
    "NUM_EPOCHS": 15,
    "MODEL_SAVE_PATH": "/home/users/gmenon/workspace/songsLyricsGenerator/src/model_artefacts/wav2vec2_demucs_en_finetuned_model.pt",
    "FINETUNE_STRATEGY": "no_freeze_deepspeed",
    "LR_SCHEDULER": "reduce_on_plateau_schedule"
}


In [2]:
import argparse
hparams = argparse.Namespace()
hparams.wav2vec2_model = 'facebook/wav2vec2-large-960h-lv60-self'
hparams.lm_model = 'bert-base-uncased' #'facebook/bart-large'
hparams.vocab_size = 40000
hparams.learning_rate = 1e-6
hparams.batch_size = 2

lyrics_finetune.run(hparams)

Namespace(batch_size=2, learning_rate=1e-05, lm_model='bert-base-uncased', vocab_size=40000, wav2vec2_model='facebook/wav2vec2-large-960h-lv60-self')


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-960h-lv60-self",
  "activation_dropout": 0.1,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": true,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dr

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

CPU Count = 4
In Stage = Fit


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.


'completed'